In [2]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

'/datastor1/zliu/mend/notebooks'

# Bio syn data

In [43]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/llama3.2-1B-eos-sft-country_syn-pretrain/common_date/base_n=100_prompt=no_w-gen_wo-icl_ice=True.xlsx")
df["is_num"] = df["is_num"].astype(float)
print("Num of rows:", len(df))
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff",]].round(2)

Num of rows: 100


,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff
count,100.00,100.00,100.00,100.0,100.00
mean,0.06,0.66,0.06,1.0,40.45
std,0.24,0.15,0.24,0.0,140.17
min,0.00,0.50,0.00,1.0,0.00
25%,0.00,0.50,0.00,1.0,1.00
50%,0.00,0.75,0.00,1.0,3.00
75%,0.00,0.75,0.00,1.0,22.25
max,1.00,1.00,1.00,1.0,1013.00


In [44]:
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff
0,specificity,0.06,0.66,0.06,1.0,40.45


In [45]:
# macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)
macro_averaging(df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff
0,specificity_0,0.06,0.66,0.06,1.0,40.45


# Country results

In [20]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/llama3.2-1B-eos-sft-bio_syn_v2-pretrain/country_syn/base_n=100_prompt=no_w-gen_wo-icl_ice=True.xlsx")
print("Num of rows:", len(df))
# df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2)

Num of rows: 500


In [21]:
# macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=metrics) * 100
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["exact_match", "llm_accuracy"]).round(2)

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM,exact_match,llm_accuracy
0,efficacy,0.25,0.63,0.0,0.26,0.25,0.4


In [22]:
macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy"]).round(2)

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,llm_accuracy
0,efficacy_compare-city,0.00,0.50,0.00,0.22
1,efficacy_compare-continent,0.00,0.50,0.00,0.22
2,efficacy_compare-country,0.00,0.50,0.00,0.22
3,efficacy_continent,0.58,0.81,0.58,0.62
4,efficacy_country,0.66,0.84,0.66,0.73


In [184]:
for q_tag, q_df in df[df["stage"] == "pre-edit"].groupby("question_tag"):
    print(q_tag + ":", sum(isinstance(x, str) for x in q_df["predicted_answer"].to_list()))

multi_hop_efficacy_q0: 999
single_hop_efficacy_q0: 802
single_hop_efficacy_q1: 744


In [45]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [34]:
save_dir = "/data/users/zliu/mend/debug_exp_output/Llama-3.2-1B-common-date-year-after-eos-sft_clm-baseline_lr=1e-05_epoch=4.0/individual_results_bio_syn_v2_text_w_qas"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

10700

In [35]:
all_df.to_excel(f"{save_dir}/../all_results_bio_syn_v2_text_w_qas.xlsx", index=False)

In [6]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [7]:
all_df.describe()

,id,answer,predicted_answer_idx,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
count,100.000000,100.000000,100.0,100.000000,100.000000,73.000000,73.000000,100.000000,100.000000,100.0,100.000000
mean,49.500000,1957.310000,0.0,0.310000,0.730000,13.493151,8.041096,0.300000,0.790000,0.0,0.438087
std,29.011492,30.458017,0.0,0.464823,0.446196,15.383183,18.859450,0.460566,0.161746,0.0,0.068536
min,0.000000,1904.000000,0.0,0.000000,0.000000,0.000000,-48.000000,0.000000,0.500000,0.0,0.285714
25%,24.750000,1932.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.418731
50%,49.500000,1954.500000,0.0,0.000000,1.000000,10.000000,0.000000,0.000000,0.750000,0.0,0.428571
75%,74.250000,1977.500000,0.0,1.000000,1.000000,20.000000,19.000000,1.000000,1.000000,0.0,0.500000
max,99.000000,2022.000000,0.0,1.000000,1.000000,61.000000,61.000000,1.000000,1.000000,0.0,0.611111


In [20]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [32]:
for i, row in nan_efficacy_df.iterrows():
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Charlotte Edwards was born?
1979 in Berlin. He started the career of Marketing Manager in 1995. In

When was the year after the year that Eric Stewart was born?
1932 in Buenos Aires

When was the year after the year that Emma Rogers was born?
1928 in Athens. 1948 she passed away.

When was the year after the year that Christina Gomez was born?
1930 in Berlin. She started the career of Chef in 1949. In 198

When was the year after the year that Laura Brown was born?
1967 in Istanbul. 1987 in Istanbul. 2002 in Istanbul. 200

When was the year after the year that Tyler Clark was born?
1904 in Moscow

When was the year after the year that Jennifer Roberts was born?
1940 in Jerusalem

When was th

In [34]:
born_question = 0
for i, row in efficacy_df.iterrows():
    born_question += "born" in row["question"]
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Brian Murphy passed away?
2005

When was the year after the year that Zoe Adams started the career of Paralegal?
1974

When was the year after the year that Victoria Phillips passed away?
2010

When was the year after the year that Madison Kelly passed away?
2003

When was the year after the year that John Reyes passed away?
1990

When was the year after the year that Eric Martinez was born?
1916

When was the year after the year that Aaron Morales started the career of Photographer?
1970

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Lucas Phillips passed away?
1959

When was the year after the year that Sofia Murphy started the career of Paralegal?
1906

When was the year after the year that Charl

In [24]:
mend_1k_n_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n-question.xlsx")

In [25]:
mend_1k_n_1_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n+1-question.xlsx")

In [26]:
len(mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1])

14

In [36]:
mend_1k_n_question_correct_ids = set((mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1]["id"].to_list()))

In [35]:
mend_1k_n_1_question_correct_ids = set(mend_1k_n_1_question_efficacy[mend_1k_n_1_question_efficacy["exact_match"] == 1]["id"].to_list())

In [37]:
mend_1k_n_question_correct_ids

{20, 28, 35, 38, 41, 46, 48, 50, 54, 56, 76, 82, 84, 89}

In [39]:
mend_1k_n_1_question_correct_ids.intersection(mend_1k_n_question_correct_ids)

{89}

In [40]:
len(mend_1k_n_1_question_correct_ids)

62